In [ ]:
from google.colab import drive
import zipfile

drive.mount('/content/drive')

# Unzip to local runtime for speed
with zipfile.ZipFile('/content/drive/MyDrive/HackfestXDatathon2026ML/Training.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

Mounted at /content/drive


In [ ]:
# ==========================================
# MASTER SCRIPT: RUN THIS CELL ONLY
# ==========================================

# 1. INSTALL & IMPORTS
!pip install -q segmentation-models-pytorch albumentations

import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm.notebook import tqdm
from google.colab import drive

# 2. SETUP & DEVICE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Using device: {DEVICE}")

# Mount Drive for saving
drive.mount('/content/drive')
save_dir = '/content/drive/MyDrive/Duality_Hackathon_Checkpoints'
os.makedirs(save_dir, exist_ok=True)

# 3. CONFIGURATION
DATA_DIR = './dataset/Offroad_Segmentation_Training_Dataset' # Corrected Path
BATCH_SIZE = 4
LR = 6e-5
EPOCHS = 15

# 4. DATASET & AUGMENTATION (The Missing Part!)
value_map = {
    0: 0, 100: 1, 200: 2, 300: 3, 500: 4,
    550: 5, 700: 6, 800: 7, 7100: 8, 10000: 9
}

def convert_mask(mask):
    arr = np.array(mask)
    new_arr = np.zeros_like(arr, dtype=np.uint8)
    for raw_value, new_value in value_map.items():
        new_arr[arr == raw_value] = new_value
    return new_arr

def get_training_augmentation():
    train_transform = [
        A.Resize(544, 960),
        A.OneOf([
            A.RandomBrightnessContrast(p=1),
            A.HueSaturationValue(p=1),
            A.RGBShift(p=1),
        ], p=0.5),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=10, shift_limit=0.1, p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
    return A.Compose(train_transform)

def get_validation_augmentation():
    test_transform = [
        A.Resize(544, 960),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
    return A.Compose(test_transform)

class DualityDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.image_dir = os.path.join(data_dir, 'Color_Images')
        self.masks_dir = os.path.join(data_dir, 'Segmentation')
        self.transform = transform
        self.data_ids = sorted(os.listdir(self.image_dir))

    def __len__(self):
        return len(self.data_ids)

    def __getitem__(self, idx):
        data_id = self.data_ids[idx]
        img_path = os.path.join(self.image_dir, data_id)
        mask_path = os.path.join(self.masks_dir, data_id)

        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path))
        mask = convert_mask(mask)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        return image, mask

# Create Loaders
print("⏳ Loading Dataset...")
train_dataset = DualityDataset(f'{DATA_DIR}/train', transform=get_training_augmentation())
val_dataset = DualityDataset(f'{DATA_DIR}/val', transform=get_validation_augmentation())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
print(f"✅ Loaded {len(train_dataset)} training images and {len(val_dataset)} validation images.")

# 5. MODEL SETUP
print("⚙️ Initializing SegFormer...")
model = smp.Segformer(
    encoder_name="mit_b1",
    encoder_weights="imagenet",
    in_channels=3,
    classes=10
)
model.to(DEVICE)

loss_fn = smp.losses.DiceLoss(mode='multiclass', from_logits=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

# 6. TRAINING LOOP
best_iou = 0.0

print(f"🚀 Starting Stabilized Training...")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    # Train
    for images, masks in tqdm(train_loader, desc=f"Epoch {epoch+1} Train"):
        images = images.to(DEVICE)
        masks = masks.long().to(DEVICE)

        optimizer.zero_grad()
        logits = model(images)
        loss = loss_fn(logits, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    # Validate (Full Loop)
    model.eval()
    tp_tot, fp_tot, fn_tot, tn_tot = 0, 0, 0, 0

    with torch.no_grad():
        for val_img, val_mask in val_loader:
            val_img = val_img.to(DEVICE)
            val_mask = val_mask.long().to(DEVICE)

            val_logits = model(val_img)

            # Calculate stats for this batch
            tp, fp, fn, tn = smp.metrics.get_stats(
                torch.argmax(val_logits, dim=1).long(),
                val_mask,
                mode='multiclass',
                num_classes=10
            )

            # Accumulate stats
            tp_tot += tp
            fp_tot += fp
            fn_tot += fn
            tn_tot += tn

    # Calculate IoU over the whole epoch using accumulated stats
    iou_score = smp.metrics.iou_score(tp_tot, fp_tot, fn_tot, tn_tot, reduction="micro")

    print(f"Epoch {epoch+1} | Loss: {train_loss/len(train_loader):.4f} | Val IoU: {iou_score:.4f}")

    # Save Best Model
    if iou_score > best_iou:
        best_iou = iou_score
        torch.save(model.state_dict(), f"{save_dir}/best_model.pth")
        print(f"  💾 Saved New Best Model to Drive! (IoU: {best_iou:.4f})")

✅ Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ Loading Dataset...
✅ Loaded 2857 training images and 317 validation images.
⚙️ Initializing SegFormer...


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


🚀 Starting Stabilized Training...


Epoch 1 Train:   0%|          | 0/715 [00:00<?, ?it/s]

Epoch 1 | Loss: 0.4036 | Val IoU: 0.7333
  💾 Saved New Best Model to Drive! (IoU: 0.7333)


...

Epoch 13 | Loss: 0.2384 | Val IoU: 0.7760
  💾 Saved New Best Model to Drive! (IoU: 0.7760)


Epoch 14 Train:   0%|          | 0/715 [00:00<?, ?it/s]

Epoch 14 | Loss: 0.2368 | Val IoU: 0.7753


Epoch 15 Train:   0%|          | 0/715 [00:00<?, ?it/s]

Epoch 15 | Loss: 0.2364 | Val IoU: 0.7760
  💾 Saved New Best Model to Drive! (IoU: 0.7760)


In [ ]:
# Unzip to local runtime for speed
with zipfile.ZipFile('/content/drive/MyDrive/HackfestXDatathon2026ML/test_public_80.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/test')

In [ ]:
import torch
import numpy as np
import cv2
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm.notebook import tqdm
from PIL import Image

# 1. CONFIG
TEST_DIR = '/content/test/test_public_80'
IMG_DIR = os.path.join(TEST_DIR, 'Color_Images')
MASK_DIR = os.path.join(TEST_DIR, 'Segmentation')
MODEL_PATH = '/content/drive/MyDrive/Duality_Hackathon_Checkpoints/best_model.pth'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Correct Class List (0-9)
class_names = [
    "Background", "Trees", "Lush Bushes", "Dry Grass", "Dry Bushes",
    "Ground Clutter", "Logs", "Rocks", "Landscape", "Sky"
]

# Map Raw Values to IDs
value_map = {0: 0, 100: 1, 200: 2, 300: 3, 500: 4, 550: 5, 700: 6, 800: 7, 7100: 8, 10000: 9}

# 2. LOAD MODEL
print("⚙️ Loading Raw Model...")
model = smp.Segformer(
    encoder_name="mit_b1",
    encoder_weights=None,
    in_channels=3,
    classes=10
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# 3. UTILS
def convert_mask(mask):
    arr = np.array(mask)
    new_arr = np.zeros_like(arr, dtype=np.uint8)
    for raw_value, new_value in value_map.items():
        new_arr[arr == raw_value] = new_value
    return new_arr

def calculate_iou(pred_mask, true_mask):
    ious = []
    # SKIP BACKGROUND (Index 0). Start from 1.
    for cls in range(1, 10):
        p = (pred_mask == cls)
        t = (true_mask == cls)
        union = (p | t).sum()
        if union == 0:
            ious.append(np.nan)
        else:
            ious.append((p & t).sum() / union)
    return np.array(ious)

# 4. RAW INFERENCE LOOP (No TTA)
transform = A.Compose([
    A.Resize(544, 960), # We MUST resize to multiple of 32 for SegFormer
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

images = sorted([f for f in os.listdir(IMG_DIR) if f.endswith('.png')])
total_ious = []

print(f"🚀 Running RAW Inference (No TTA) on {len(images)} images...")

for img_name in tqdm(images):
    img_path = os.path.join(IMG_DIR, img_name)
    mask_path = os.path.join(MASK_DIR, img_name)

    # Load
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    H, W = image.shape[:2]

    # Preprocess
    augmented = transform(image=image)
    img_tensor = augmented['image'].unsqueeze(0).to(DEVICE)

    # Predict (Single Pass)
    with torch.no_grad():
        logits = model(img_tensor)
        # Remap Background(0) to Landscape(8)
        probs = torch.softmax(logits, dim=1)
        probs[:, 8, :, :] += probs[:, 0, :, :]
        probs[:, 0, :, :] = 0
        pred_mask = torch.argmax(probs, dim=1).cpu().numpy()[0]

    # Resize Back
    pred_mask = cv2.resize(pred_mask.astype('uint8'), (W, H), interpolation=cv2.INTER_NEAREST)

    # Evaluate
    if os.path.exists(mask_path):
        gt_mask = np.array(Image.open(mask_path))
        gt_mask = convert_mask(gt_mask)
        total_ious.append(calculate_iou(pred_mask, gt_mask))

# 5. PRINT CORRECTED SCORES
all_ious = np.array(total_ious)
mean_class_iou = np.nanmean(all_ious, axis=0)

print("\n" + "="*40)
print(f"📊 RAW SCORES (No TTA)")
print("="*40)
# ZIP CORRECTLY: Skip 'Background' in names list too
for name, iou in zip(class_names[1:], mean_class_iou):
    print(f"{name:<20}: {iou:.4f}")

print("-" * 40)
print(f"🏆 RAW MEAN IoU: {np.nanmean(mean_class_iou):.4f}")

⚙️ Loading Raw Model...
🚀 Running RAW Inference (No TTA) on 801 images...


  0%|          | 0/801 [00:00<?, ?it/s]


📊 RAW SCORES (No TTA)
Trees               : 0.3502
Lush Bushes         : 0.0032
Dry Grass           : 0.4788
Dry Bushes          : 0.3883
Ground Clutter      : 0.0000
Logs                : 0.0000
Rocks               : 0.0481
Landscape           : 0.6739
Sky                 : 0.9721
----------------------------------------
🏆 RAW MEAN IoU: 0.3238


In [ ]:
# ==========================================
# MASTER SCRIPT: DEEPLABV3+ (ResNet34)
# ==========================================
!pip install -q segmentation-models-pytorch albumentations

import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm.notebook import tqdm
from google.colab import drive

# 1. SETUP
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Using device: {DEVICE}")

drive.mount('/content/drive')
save_dir = '/content/drive/MyDrive/Duality_Hackathon_Checkpoints/DeepLab'
os.makedirs(save_dir, exist_ok=True)

# 2. CONFIGURATION
DATA_DIR = './dataset/Offroad_Segmentation_Training_Dataset'
# DeepLab ResNet34 is efficient. We can try Batch Size 8 for smoother training.
# If it crashes (OOM), change back to 4.
BATCH_SIZE = 8
LR = 1e-4
EPOCHS = 15

# 3. DATA & AUGMENTATION
value_map = {0: 0, 100: 1, 200: 2, 300: 3, 500: 4, 550: 5, 700: 6, 800: 7, 7100: 8, 10000: 9}

def convert_mask(mask):
    arr = np.array(mask)
    new_arr = np.zeros_like(arr, dtype=np.uint8)
    for raw_value, new_value in value_map.items():
        new_arr[arr == raw_value] = new_value
    return new_arr

def get_training_augmentation():
    return A.Compose([
        A.Resize(544, 960),
        # Slightly more aggressive geometry for CNNs
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=10, shift_limit=0.1, p=0.5),
        # Color Jitter is CRITICAL for the "Novel Environment"
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=1),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1),
            A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=1),
        ], p=0.8),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

def get_validation_augmentation():
    return A.Compose([
        A.Resize(544, 960),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

class DualityDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.image_dir = os.path.join(data_dir, 'Color_Images')
        self.masks_dir = os.path.join(data_dir, 'Segmentation')
        self.transform = transform
        self.data_ids = sorted(os.listdir(self.image_dir))

    def __len__(self): return len(self.data_ids)

    def __getitem__(self, idx):
        data_id = self.data_ids[idx]
        img_path = os.path.join(self.image_dir, data_id)
        mask_path = os.path.join(self.masks_dir, data_id)
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path))
        mask = convert_mask(mask)
        if self.transform:
            aug = self.transform(image=image, mask=mask)
            image, mask = aug['image'], aug['mask']
        return image, mask

# Loaders
train_dataset = DualityDataset(f'{DATA_DIR}/train', transform=get_training_augmentation())
val_dataset = DualityDataset(f'{DATA_DIR}/val', transform=get_validation_augmentation())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# 4. MODEL: DEEPLABV3+ (The Change)
print("⚙️ Initializing DeepLabV3+ (ResNet34)...")
model = smp.DeepLabV3Plus(
    encoder_name="resnet34",        # Reliable CNN backbone
    encoder_weights="imagenet",     # Pre-trained
    in_channels=3,
    classes=10
)
model.to(DEVICE)

# 5. LOSS & OPTIMIZER
# We stick with DiceLoss because IoU is the metric.
loss_fn = smp.losses.DiceLoss(mode='multiclass', from_logits=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# 6. TRAINING LOOP
best_iou = 0.0
print(f"🚀 Starting DeepLab Training...")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    # Train
    for images, masks in tqdm(train_loader, desc=f"Epoch {epoch+1} Train"):
        images = images.to(DEVICE)
        masks = masks.long().to(DEVICE)

        optimizer.zero_grad()
        logits = model(images)
        loss = loss_fn(logits, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    # Validate
    model.eval()
    tp_tot, fp_tot, fn_tot, tn_tot = 0, 0, 0, 0

    with torch.no_grad():
        for val_img, val_mask in val_loader:
            val_img = val_img.to(DEVICE)
            val_mask = val_mask.long().to(DEVICE)

            val_logits = model(val_img)

            tp, fp, fn, tn = smp.metrics.get_stats(
                torch.argmax(val_logits, dim=1).long(),
                val_mask, mode='multiclass', num_classes=10
            )
            tp_tot += tp; fp_tot += fp; fn_tot += fn; tn_tot += tn

    iou_score = smp.metrics.iou_score(tp_tot, fp_tot, fn_tot, tn_tot, reduction="micro")

    print(f"Epoch {epoch+1} | Loss: {train_loss/len(train_loader):.4f} | Val IoU: {iou_score:.4f}")

    # SAVE BEST
    if iou_score > best_iou:
        best_iou = iou_score
        torch.save(model.state_dict(), f"{save_dir}/deeplab_best.pth")
        print(f"  💾 Saved DeepLab Best! (IoU: {best_iou:.4f})")

✅ Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⚙️ Initializing DeepLabV3+ (ResNet34)...
🚀 Starting DeepLab Training...


Epoch 1 Train:   0%|          | 0/358 [00:00<?, ?it/s]

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 256, 1, 1])

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import pandas as pd

# 1. SETUP
# Path to your validation segmentation folder
MASK_DIR = '/content/test/test_public_80/Segmentation'

# The mapping used in your training
value_map = {0: 0, 100: 1, 200: 2, 300: 3, 500: 4, 550: 5, 700: 6, 800: 7, 7100: 8, 10000: 9}
class_names = [
    "Background", "Trees", "Lush Bushes", "Dry Grass", "Dry Bushes",
    "Ground Clutter", "Logs", "Rocks", "Landscape", "Sky"
]

# 2. ANALYSIS
def analyze_dataset_presence(mask_dir):
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith('.png')]
    # Counter for how many images contain at least one pixel of a class
    image_presence = np.zeros(10)
    # Total pixel count per class
    total_pixels = np.zeros(10)

    print(f"🔍 Scanning {len(mask_files)} masks for class presence...")

    for f in tqdm(mask_files):
        mask_path = os.path.join(mask_dir, f)
        # Load and convert to class IDs 0-9
        mask_raw = np.array(Image.open(mask_path))
        mask_ids = np.zeros_like(mask_raw, dtype=np.uint8)
        for raw_val, class_id in value_map.items():
            mask_ids[mask_raw == raw_val] = class_id

        # Check presence and count pixels
        unique_classes, counts = np.unique(mask_ids, return_counts=True)
        for cls, count in zip(unique_classes, counts):
            image_presence[cls] += 1
            total_pixels[cls] += count

    # Create Report
    report = []
    for i in range(10):
        report.append({
            "Class ID": i,
            "Class Name": class_names[i],
            "Present in # Images": int(image_presence[i]),
            "% of Images": f"{(image_presence[i]/len(mask_files))*100:.1f}%",
            "Total Pixel Count": int(total_pixels[i])
        })

    return pd.DataFrame(report)

# 3. RUN
df = analyze_dataset_presence(MASK_DIR)
print("\n" + "="*60)
print("📊 CLASS PRESENCE REPORT (Validation Set)")
print("="*60)
print(df.to_string(index=False))

🔍 Scanning 801 masks for class presence...


  0%|          | 0/801 [00:00<?, ?it/s]


📊 CLASS PRESENCE REPORT (Validation Set)
 Class ID     Class Name  Present in # Images % of Images  Total Pixel Count
        0     Background                    0        0.0%                  0
        1          Trees                  788       98.4%            1118906
        2    Lush Bushes                  542       67.7%               6327
        3      Dry Grass                  801      100.0%           72000224
        4     Dry Bushes                  801      100.0%           12447465
        5 Ground Clutter                    0        0.0%                  0
        6           Logs                    0        0.0%                  0
        7          Rocks                  801      100.0%           75639480
        8      Landscape                  801      100.0%          179434762
        9            Sky                  801      100.0%           74591236


In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import pandas as pd

# 1. SETUP - POINT TO TRAINING DATA
TRAIN_MASK_DIR = './dataset/Offroad_Segmentation_Training_Dataset/train/Segmentation'

value_map = {0: 0, 100: 1, 200: 2, 300: 3, 500: 4, 550: 5, 700: 6, 800: 7, 7100: 8, 10000: 9}
class_names = ["Background", "Trees", "Lush Bushes", "Dry Grass", "Dry Bushes",
               "Ground Clutter", "Logs", "Rocks", "Landscape", "Sky"]

# 2. ANALYSIS FUNCTION
def analyze_train_presence(mask_dir):
    if not os.path.exists(mask_dir):
        print(f"❌ Error: Path {mask_dir} not found!")
        return None

    mask_files = [f for f in os.listdir(mask_dir) if f.endswith('.png')]
    image_presence = np.zeros(10)
    total_pixels = np.zeros(10)

    print(f"🔍 Scanning {len(mask_files)} TRAINING masks...")

    for f in tqdm(mask_files):
        mask_path = os.path.join(mask_dir, f)
        mask_raw = np.array(Image.open(mask_path))
        mask_ids = np.zeros_like(mask_raw, dtype=np.uint8)
        for raw_val, class_id in value_map.items():
            mask_ids[mask_raw == raw_val] = class_id

        unique_classes, counts = np.unique(mask_ids, return_counts=True)
        for cls, count in zip(unique_classes, counts):
            image_presence[cls] += 1
            total_pixels[cls] += count

    report = []
    for i in range(10):
        report.append({
            "Class ID": i,
            "Class Name": class_names[i],
            "Present in # Images": int(image_presence[i]),
            "Total Pixels": int(total_pixels[i])
        })
    return pd.DataFrame(report)

# 3. RUN
df_train = analyze_train_presence(TRAIN_MASK_DIR)
if df_train is not None:
    print("\n📊 TRAINING DATA CLASS DISTRIBUTION")
    print(df_train.to_string(index=False))

🔍 Scanning 2857 TRAINING masks...


  0%|          | 0/2857 [00:00<?, ?it/s]


📊 TRAINING DATA CLASS DISTRIBUTION
 Class ID     Class Name  Present in # Images  Total Pixels
        0     Background                 1010      41585811
        1          Trees                 2247      52331525
        2    Lush Bushes                 2806      87892776
        3      Dry Grass                 2826     279430843
        4     Dry Bushes                  914      16268713
        5 Ground Clutter                 2855      65082995
        6           Logs                 1685       1153995
        7          Rocks                 2823      17743187
        8      Landscape                 2855     362120221
        9            Sky                 2587     557458734


In [ ]:
# RUN THIS ON YOUR SEGFORMER WEIGHTS
!pip install -q segmentation-models-pytorch albumentations
import torch
import numpy as np
import segmentation_models_pytorch as smp

# Ensure this points to your SegFormer .pth file
SEGFORMER_PATH = '/content/drive/MyDrive/Duality_Hackathon_Checkpoints/best_model.pth'

# Define architecture to match your weights
model = smp.Segformer(
    encoder_name="mit_b1",
    in_channels=3,
    classes=10
)
model.load_state_dict(torch.load(SEGFORMER_PATH, map_location=torch.device('cpu')))
model.to('cpu').eval()


# Use the 'calculate_smart_iou' logic from before:
# 1. Skip Background (0)
# 2. Skip Ground Clutter (5) and Logs (6) since they are 0 in your scan.
# 3. Focus on Trees, Bushes, Grass, Rocks, Landscape, Sky.

Segformer(
  (encoder): MixVisionTransformerEncoder(
    (patch_embed1): OverlapPatchEmbed(
      (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed2): OverlapPatchEmbed(
      (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed3): OverlapPatchEmbed(
      (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed4): OverlapPatchEmbed(
      (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (block1): Sequential(
      (0): Block(
        (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (q): Linear(in_featur